# Creating interactive maps with Leaflet and Folium 

## Authenticating to Google Earth Engine

In [1]:
%%bash
earthengine authenticate --quiet

Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API

Paste the following address into a web browser:

    https://accounts.google.com/o/oauth2/auth?client_id=517222506229-vsmmajv00ul0bs7p89v5m89qs8eb9359.apps.googleusercontent.com&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fearthengine+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdevstorage.full_control&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&response_type=code

On the web page, please authorize access to your Earth Engine account and copy the authentication code. Next authenticate with the following command:

    earthengine authenticate --authorization-code=PLACE_AUTH_CODE_HERE



Once you have obtained an authorization code from the previous step, paste the code into the following cell and run it:

In [3]:
%%bash
earthengine authenticate --authorization-code=4/sAFDcG351nLy1eBlGoGNhdQb3XI2OD4p-tnABgaC-Aq1bX_VhfaALNk

Running command using Cloud API.  Set --no-use_cloud_api to go back to using the API


Successfully saved authorization token.


## Import Python packages

In [4]:
from oauth2client import crypt
# import packages
import pandas as pd
import json
import sys
import ee 

ee.Initialize()

import folium
print(folium.__version__)
sys.path.append('..')
print (folium.__file__)
print (folium.__version__)

0.10.0
/opt/conda/lib/python3.7/site-packages/folium/__init__.py
0.10.0


### Create definition files for working with Google Earth Engine

In [5]:
def folium_gee_map(image,vis_params=None,folium_kwargs={}):
    """
    Function to view Google Earth Engine tile layer as a Folium map.
    
    Parameters
    ----------
    image : Google Earth Engine Image.
    vis_params : Dict with visualization parameters.
    folium_kwargs : Keyword args for Folium Map.
    """
    
    # Get the MapID and Token after applying parameters
    image_info = image.getMapId(vis_params)
    mapid = image_info['mapid']
    token = image_info['token']
    folium_kwargs['attr'] = ('Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a> ')
    folium_kwargs['tiles'] = "https://earthengine.googleapis.com/map/%s/{z}/{x}/{y}?token=%s"%(mapid,token)
    
    return folium.Map(**folium_kwargs)

def folium_gee_layer(folium_map,image,vis_params=None,folium_kwargs={}):
    """
    Function to add Google Earch Engine tile layer as a Folium layer.
    
    Parameters
    ----------
    folium_map : Folium map to add tile to.
    image : Google Earth Engine Image.
    vis_params : Dict with visualization parameters.
    folium_kwargs : Keyword args for Folium Map.
    """
    
    # Get the MapID and Token after applying parameters
    image_info = image.getMapId(vis_params)
    mapid = image_info['mapid']
    token = image_info['token']
    folium_kwargs['attr'] = ('Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a> ')
    folium_kwargs['tiles'] = "https://earthengine.googleapis.com/map/%s/{z}/{x}/{y}?token=%s"%(mapid,token)
    
    layer = folium.TileLayer(**folium_kwargs)
    layer.add_to(folium_map)


### Initiate your connection to the CyVerse DataStore with iRODS iCommands

This little script _MUST_ be run on a secure connection, else your password will be visible to anyone monitoring your work.

If you're not comfortable entering your CyVerse password in the notebook, you can open a terminal session and type `iinit`

You will be asked for your password there.

Now, we want to copy our data to a new scratch directory. If your user does not own the directory you will have to also use `chmod` to change ownership.

```
sudo chown $USER:iplant-everone /scratch/ -R
```

Note, you will have to use `sudo` for directories your username does not own.

### Import Santa Rita Experimental Range Data boundary KMLs from DataStore

In [8]:
%%bash
mkdir -p /home/jovyan/NEON_data_institute_2018/2017_Campaign/SRER/L1/

In [9]:
%%bash
iget -Pbrvf /iplant/home/shared/NEON_data_institute_2018/2017_Campaign/SRER/L1/LasBoundary  /home/jovyan/NEON_data_institute_2018/2017_Campaign/SRER/L1/

D- /home/jovyan/NEON_data_institute_2018/2017_Campaign/SRER/L1//LasBoundary :
0/128 -  0.00% of files done   0.000/20.066 MB -  0.00% of file sizes done
Processing NEON_D14_SRER_DP1_L001-1_2017082413_unclassified_point_cloud_boundary.kml - 0.212 MB   2019-10-14.01:38:14
   NEON_D14_SRER_DP1_L001-1_       0.212 MB | 1.501 sec | 0 thr |  0.141 MB/s
1/128 -  0.78% of files done   0.212/20.066 MB -  1.06% of file sizes done
Processing NEON_D14_SRER_DP1_L001-2_2017082419_unclassified_point_cloud_boundary.kml - 0.249 MB   2019-10-14.01:38:15
   NEON_D14_SRER_DP1_L001-2_       0.249 MB | 0.332 sec | 0 thr |  0.751 MB/s
2/128 -  1.56% of files done   0.462/20.066 MB -  2.30% of file sizes done
Processing NEON_D14_SRER_DP1_L001-3_2017082514_unclassified_point_cloud_boundary.kml - 0.063 MB   2019-10-14.01:38:15
   NEON_D14_SRER_DP1_L001-3_       0.063 MB | 1.665 sec | 0 thr |  0.038 MB/s
3/128 -  2.34% of files done   0.524/20.066 MB -  2.61% of file sizes done
Processing NEON_D14_SRER_DP1_L002-

##### Convert the .kml to a .geojson file.

In [10]:
import kml2geojson
kml2geojson.main.convert("/home/jovyan/NEON_data_institute_2018/2017_Campaign/SRER/L1/LasBoundary/full_boundary.kml", "/home/jovyan//NEON_data_institute_2018/2017_Campaign/SRER/L1/LasBoundary/")

In [11]:
# import geo json data
import json
full_boundary = json.load(open('/home/jovyan//NEON_data_institute_2018/2017_Campaign/SRER/L1/LasBoundary/full_boundary.geojson'))

# Plot Leaflet Map over the SRER NEON site 

We are also adding the NASA SRTM shuttle data, Landsat 8 TOA, and Guillermo Ponce-Campos collection of the NEON Orthophotography.

In [12]:
# Get an area to look at
lat = 31.800
lon = -110.865
zoom_start=11
zoom_max=24

# Folium Base Maps https://python-visualization.github.io/folium/quickstart.html
m = folium.Map(location=[lat, lon], tiles='Stamen Terrain', zoom_start=zoom_start, max_zoom=zoom_max)

# Add polygon outline of SRER
full_boundary = json.load(open('/home/jovyan/NEON_data_institute_2018/2017_Campaign/SRER/L1/LasBoundary/full_boundary.geojson'))
folium.GeoJson(full_boundary,name="Santa Rita Experimental Range Boundary").add_to(m)

# Create a reference to the LANDSAT 8 Tier 1 Surface Reflectance image collection and set the reference period to 2017
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
# Filter the LANDSAT collection down to a eight month period
filtered = l8.filterDate('2017-07-01', '2017-08-31').filterMetadata('CLOUD_COVER','less_than',2);
# Use the mosaic reducer, to select the most recent pixel in areas of overlap
l8_image = filtered.median()
l8_vis_params = {'min': 0, 'max':2000, 'bands':'B4,B3,B2'}
folium_gee_layer(m,l8_image,l8_vis_params,folium_kwargs={'overlay':True,'name':'LANDSAT 8 SR Tier 1'})

# Create a reference to the 2017 Planet Labs PlanetScope Surface Reflectance Data
planet = ee.ImageCollection('users/samapriya/planet-impact/neon_ps4b_sr')
planet_image = planet.median()
planet_vis_params = {'min':1,'max':2000,'gamma':1,'bands':'b3,b2,b1'}
folium_gee_layer(m,planet_image,planet_vis_params,folium_kwargs={'overlay':True,'name':'PlanetScope Surface Reflectance 2017'})

# Create a reference to the 2017 SRER NEON orthophotography image collection
neon = ee.ImageCollection('users/gponce/usda_ars/image_collections/neon_srer_2017_rgb')
neon_image = neon.median()
neon_vis_params = {'min':0, 'max':255, 'bands':'b1,b2,b3'}
folium_gee_layer(m,neon_image,neon_vis_params,folium_kwargs={'overlay':True,'name':'NEON AOP 10cm 2017'})

# Create a reference to the 2019 sUAS orthophotography imagery
suas = ee.Image('users/gponce/usda_ars/assets/images/aes/srer/suas/2019/ortho_spring_2019_1cm')
folium_gee_layer(m,suas,folium_kwargs={'overlay':True,'name':'2019 DJI Phantom RTK'})

m.add_child(folium.LayerControl())
m